# **Sweep Notebook**
This notebook is easily customizable for easier sweeping and finetuning rather than having messy dev sweeps that all look different. This will be the go to for sweeping.

## **Import Libraries**

In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np


# --- Global Black-Scholes parameters and grids ---
r = 0.05          # Risk-free rate
sigma = 0.2       # Volatility
K = 100           # Strike price
T = 1.0           # Time to maturity (in years)
S_max = 250       # Max stock price in spatial domain
S_min = 0         # Min stock price
N = 500           # Number of spatial grid points

# Create spatial grid
S = torch.linspace(S_min, S_max, N).view(-1, 1).requires_grad_()
t = torch.linspace(0, T, N).view(-1, 1).requires_grad_()

## **Model**

In [ ]:
class blackScholesPINN(nn.Module):
    def __init__(self, input_dim=2, output_dim=1, hidden_layers=4, neurons_per_layer=64, activation_fn=nn.Tanh()):
        super().__init__()
        layers = []

        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(activation_fn)

        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(activation_fn)

        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


def initialize_weights(model, method='xavier'):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if method == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif method == 'kaiming':
                nn.init.kaiming_uniform_(m.weight)
            elif method == 'normal':
                nn.init.normal_(m.weight, mean=0.0, std=0.1)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

## **Losses**

In [ ]:
def pde_loss(model, S, t):
    S.requires_grad_(True)
    t.requires_grad_(True)
    X = torch.cat((S, t), dim=1)
    V = model(X)

    V_t = torch.autograd.grad(V, t, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_S = torch.autograd.grad(V, S, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_SS = torch.autograd.grad(V_S, S, grad_outputs=torch.ones_like(V_S), create_graph=True)[0]

    residual = V_t + 0.5 * sigma**2 * S**2 * V_SS + r * S * V_S - r * V
    return torch.mean(residual.pow(2))


def boundary_loss(model, t):
    S0 = torch.zeros_like(t)
    S_high = torch.full_like(t, S_max)

    bc_low = model(torch.cat((S0, t), dim=1))
    bc_high = model(torch.cat((S_high, t), dim=1))
    expected_high = S_max - K * torch.exp(-r * (T - t))

    return torch.mean(bc_low.pow(2)) + torch.mean((bc_high - expected_high).pow(2))


def initial_loss(model, S):
    t0 = torch.zeros_like(S)
    X0 = torch.cat((S, t0), dim=1)

    V_pred = model(X0)
    V_true = torch.clamp(S - K, min=0.0)

    return torch.mean((V_pred - V_true).pow(2))

## **Configuration**

In [ ]:
epochs = 7500
num_sweeps = 20

config_flags = {
    'sweep_hidden_layers': False,
    'sweep_neurons': False,
    'sweep_activation': False,
    'sweep_init': False,
    'sweep_learning_rate': False,
    'sweep_loss_weights': True,
    'sweep_method': 'bayes'  # Choose from: 'random', 'bayes', 'grid'
}

# --------------------------
# DEFAULT VALUES (used if not swept)
# --------------------------
fixed_defaults = {
    'hidden_layers': 6,
    'neurons_per_layer': 32,
    'activation': 'relu',
    'init_method': 'xavier',
    'initial_lr': 0.005,
    'pde_weight_scale': 14.46,
    'bc_weight_scale': 0.411,
    'ic_weight_scale': 0.086
}

# --------------------------
# DYNAMIC SWEEP CONFIG BUILDER
# --------------------------
def generate_sweep_config(flags, defaults):
    sweep_config = {
        'method': flags['sweep_method'],
        'metric': {
            'name': 'final_total_loss',
            'goal': 'minimize'
        },
        'parameters': {}
    }

    if flags['sweep_hidden_layers']:
        sweep_config['parameters']['hidden_layers'] = {
            'values': [2, 4, 6, 8]
        }
    else:
        sweep_config['parameters']['hidden_layers'] = {
            'value': defaults['hidden_layers']
        }

    if flags['sweep_neurons']:
        sweep_config['parameters']['neurons_per_layer'] = {
            'values': [32, 64, 128]
        }
    else:
        sweep_config['parameters']['neurons_per_layer'] = {
            'value': defaults['neurons_per_layer']
        }

    if flags['sweep_activation']:
        sweep_config['parameters']['activation'] = {
            'values': ['tanh', 'relu', 'silu']
        }
    else:
        sweep_config['parameters']['activation'] = {
            'value': defaults['activation']
        }

    if flags['sweep_init']:
        sweep_config['parameters']['init_method'] = {
            'values': ['xavier', 'kaiming', 'normal']
        }
    else:
        sweep_config['parameters']['init_method'] = {
            'value': defaults['init_method']
        }

    if flags['sweep_learning_rate']:
        sweep_config['parameters']['initial_lr'] = {
            'min': 0.0001,
            'max': 0.01,
            'distribution': 'log_uniform_values'
        }
    else:
        sweep_config['parameters']['initial_lr'] = {
            'value': defaults['initial_lr']
        }

    if flags['sweep_loss_weights']:
        sweep_config['parameters']['pde_weight_scale'] = {
            'min': 5.0, 'max': 15.0, 'distribution': 'log_uniform_values'
        }
        sweep_config['parameters']['bc_weight_scale'] = {
            'min': 0.1, 'max': 3.5, 'distribution': 'log_uniform_values'
        }
        sweep_config['parameters']['ic_weight_scale'] = {
            'min': 0.05, 'max': 3.5, 'distribution': 'log_uniform_values'
        }
    else:
        sweep_config['parameters']['pde_weight_scale'] = {'value': defaults['pde_weight_scale']}
        sweep_config['parameters']['bc_weight_scale'] = {'value': defaults['bc_weight_scale']}
        sweep_config['parameters']['ic_weight_scale'] = {'value': defaults['ic_weight_scale']}

    return sweep_config

## **Sweep Function**

In [ ]:
def train_pinn_sweep():
    wandb.init()
    config = wandb.config

    # Set up activation
    activation_map = {
        'tanh': nn.Tanh(),
        'relu': nn.ReLU(),
        'silu': nn.SiLU()
    }
    activation_fn = activation_map.get(config.activation, nn.Tanh())

    # Build model
    model = blackScholesPINN(
        hidden_layers=config.hidden_layers,
        neurons_per_layer=config.neurons_per_layer,
        activation_fn=activation_fn
    )

    initialize_weights(model, config.init_method)

    # Fixed hyperparameters
    lr = config.initial_lr
    pde_weight = config.pde_weight_scale
    bc_weight = config.bc_weight_scale
    ic_weight = config.ic_weight_scale

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=300)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        V = model(torch.cat((S, t), dim=1))  # assumes S and t are defined globally or imported

        pde_l = pde_loss(model, S, t)
        bc_l = boundary_loss(model, t)
        ic_l = initial_loss(model, S)

        total_loss = pde_weight * pde_l + bc_weight * bc_l + ic_weight * ic_l
        total_loss.backward()
        optimizer.step()
        scheduler.step(total_loss)

        wandb.log({
            "epoch": epoch,
            "total_loss": total_loss.item(),
            "pde_loss": pde_l.item(),
            "bc_loss": bc_l.item(),
            "ic_loss": ic_l.item(),
            "learning_rate": optimizer.param_groups[0]['lr']
        })

        if epoch % 500 == 0:
            print(f"[{epoch}] Total: {total_loss.item():.5f}, PDE: {pde_l.item():.5f}, BC: {bc_l.item():.5f}, IC: {ic_l.item():.5f}")

    wandb.log({
        "final_total_loss": total_loss.item()
    })
    wandb.finish()

## **Instantiate Sweep**

In [ ]:
sweep_config = generate_sweep_config(config_flags, fixed_defaults)
sweep_id = wandb.sweep(sweep_config, project="Black-Scholes-Architecture-Sweep")

print(f"Sweep ID: {sweep_id}")
print("wandb agent " + sweep_id)
print(f"wandb.agent('{sweep_id}', function=train_pinn_sweep, count={num_sweeps})")

### **Paste Function Call**

In [ ]:
#wandb.agent('ie5y2esn', function=train_pinn_sweep, count=20)

## **Actually Train**

In [ ]:
def train_pinn():
        model = blackScholesPINN(
        hidden_layers=config.hidden_layers,
        neurons_per_layer=config.neurons_per_layer,
        activation_fn=activation_fn
    )